# Лабораторная работа по ПГП (№6)

## 2 задача
### Вычисление множества ближайших соседей каждой точки в облаке в NUMBA

#### Стандартная реализация с помощью Sklearn

In [1]:
import numpy as np
import sklearn as skl
from sklearn import datasets
from sklearn.neighbors import NearestNeighbors

In [2]:
#набор точек
z, y = skl.datasets.make_blobs(n_samples=10000, centers=2, n_features=2, random_state=0)
y = np.zeros(10000)
x = np.zeros(10000)
# Создаём массив из x и y
for i in range(len(z)):
    x[i],y[i] = z[i];
c = np.concatenate((x, y))
c = np.reshape(c, (-1, 2), order = 'F')
c

array([[ 0.80901751,  3.69975424],
       [ 1.17304017,  5.27341106],
       [ 2.50904929,  5.7731461 ],
       ...,
       [ 2.36680405,  2.66243872],
       [ 2.06682596, -0.03600395],
       [ 1.4159006 ,  3.79037687]])

In [3]:
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(c)
distances, indices = nbrs.kneighbors(c)
print(f'{indices} \n {distances}')

[[   0 4349]
 [   1 8914]
 [   2 1224]
 ...
 [9997 4194]
 [9998 3886]
 [9999 6510]] 
 [[0.         0.03220988]
 [0.         0.01347505]
 [0.         0.046338  ]
 ...
 [0.         0.03492665]
 [0.         0.01349694]
 [0.         0.02582222]]


#### Параллельная реализация с Numba, Vincently и Sklearn

Формула Виценти (Vicenty) позволяет вычислить расстояние от точки до точки на эллипсоиде с очень высокой точностью. Широко используется для географических задач.

In [4]:
import numba
from sklearn.neighbors import BallTree
from numba import njit, prange, guvectorize
from cuda_friendly_vincenty import vincenty

In [5]:
c_nb = np.array(c, dtype=np.float32)
c_nb
#np.shape(c_nb)

array([[ 0.8090175 ,  3.6997542 ],
       [ 1.1730402 ,  5.2734113 ],
       [ 2.5090492 ,  5.773146  ],
       ...,
       [ 2.3668041 ,  2.6624386 ],
       [ 2.0668259 , -0.03600395],
       [ 1.4159006 ,  3.790377  ]], dtype=float32)

In [6]:
compiled_vincenty = njit(vincenty)

# Нужно изменить формат вводных данных, чтобы они подошли к формату compiled_vincenty:
@numba.njit(fastmath=True)
#(fastmath=True, debug=True)
def compiled_vincenty_changed_args(point1, point2):
    return compiled_vincenty(point1[0], point1[1], point2[0], point2[1])

nbrs_nb = BallTree(c_nb, leaf_size=5, metric=compiled_vincenty_changed_args)

# Поиск двух ближайших точек, включая себя:
distances_nb, indices_nb = nbrs_nb.query(c_nb, k=2)

# данная версия измеряет по-другому, потому для того, чтобы выглядело одинаково:
distances_nb[:] = [x / 100000 for x in distances_nb]

print(f'{indices_nb} \n {distances_nb}')

[[   0 4349]
 [   1 8914]
 [   2 1224]
 ...
 [9997 4194]
 [9998 3886]
 [9999 6510]] 
 [[0.         0.0357732 ]
 [0.         0.01492741]
 [0.         0.05124457]
 ...
 [0.         0.03882857]
 [0.         0.01492474]
 [0.         0.0286251 ]]


#### Замер времени

In [7]:
#CPU (Numpy)
%timeit -n100 NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(c)

3.37 ms ± 8.72 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
#GPU (Numba)
%timeit -n100 BallTree(c_nb, leaf_size=5, metric=compiled_vincenty_changed_args)

61 ms ± 187 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
